In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import re
import time



In [4]:
def highlight(element, seconds = 5):
    """Highlights (blinks) a Selenium Webdriver element"""
    driver = element._parent
    def apply_style(s):
        driver.execute_script("arguments[0].setAttribute('style', arguments[1]);",element, s)
        
    original_style = element.get_attribute('style')
    apply_style("background: yellow; border: 5px solid red;")
    time.sleep(seconds)
    apply_style(original_style)
    

driver = webdriver.Firefox()

In [5]:
searchURL = 'https://www.sreality.cz/hledani/prodej/byty/jihomoravsky-kraj?region=m%C4%9Bstsk%C3%A1%20%C4%8D%C3%A1st%20Lesn%C3%A1&region-id=14643&region-typ=ward'
#searchURL = "https://www.sreality.cz/hledani/prodej/byty/blansko,brno-venkov,brno?plocha-od=150&plocha-do=10000000000"

driver.get(searchURL)
#in program - we need wait 10-15 seconds to fully load the page (time.sleep(10))

In [12]:
#number of found offers: "Nalezeno 26 inzerátů"
#/html/body/div[2]/div[1]/div[2]/div[2]/div[4]/div/div/div/div/div[3]/div/div[23]/p/span[2]
noffers=int(driver.find_element_by_xpath("//div[2]/div[2]/div[4]/div/div/div/div/div[3]/div/div[22]/p/span[2]").text)
noffers

                               

22

In [13]:
# new dataframe
flats = pd.DataFrame(columns=['Title', 'Price', 'Link', 'MyRate', 'State', 'Comment'])

#read csv
#flats = pd.read_csv('sreality_2019-01-19.csv', index_col =0)
#flats.head()

In [14]:
pages = noffers // 20 + (noffers % 20 >0)
#note // is integer division (no remainders)
#note % is modulo


pages

2

In [15]:
#size /html/body/div[2]/div[1]/div[2]/div[2]/div[4]/div/div/div/div/div[3]/div/div[3]/div/div/span/h2/a/span
#location /html/body/div[2]/div[1]/div[2]/div[2]/div[4]/div/div/div/div/div[3]/div/div[3]/div/div/span/span[1]
#link: /html/body/div[2]/div[1]/div[2]/div[2]/div[4]/div/div/div/div/div[3]/div/div[4]/div/div/span/h2/a
#price /html/body/div[2]/div[1]/dbiv[2]/div[2]/div[4]/div/div/div/div/div[3]/div/div[21]/div/div/span/span[2]/span

#State - set initial value to 'old'
flats.loc[:,'State'] = 'Old'

#add offers to dataframe flats
index = 0

#for every page
for page in range(1,pages+1):
    
    #load page
    if page > 1:
        URLNextPage = searchURL + "&strana=" + str(page)
        print('loading: ' + URLNextPage)
        driver.get(URLNextPage)
    offers  = driver.find_elements_by_xpath("//div[4]/div/div/div/div/div[3]/div/div[*]")
    time.sleep(3)

    #find number of offers on the page
    #prange = driver.find_element_by_css_selector('p.info.ng-binding span.numero.ng-binding').text
    #prange_split = prange.split('–')
    #page_offers = int(prange_split[1]) - int(prange_split[0])
    
    page_offers = len(driver.find_elements_by_css_selector('div.property'))
    
    print("Page offers:" + str(page_offers))
    
    #search through offers
    for i in range(0,page_offers):
        index = index+1
        highlight(offers[i], np.random.randint(0,5))
    
        #text parsing
        size = re.sub("bytu","",re.sub("Prodej","",offers[i].find_element_by_xpath("div/div/span/h2/a/span").text))
        location = re.sub("ulice","Ul",offers[i].find_element_by_xpath("div/div/span/span[1]").text)
        price = offers[i].find_element_by_xpath("div/div/span/span[2]/span").text
        link  = offers[i].find_element_by_xpath("div/div/span/h2/a").get_attribute("href")

        print(str(page) + ';'+str(i)+'  ;;;' + ''.join((location,size, price)) +'. '+ link, end = '')
        search = re.search(r'\/(\d{5,})\s*$',link)
        #continue only if search was successful
        if search:
            flatid = int(search.group(1))
    
            #add to flats
            if flatid in flats.index:
                flats.loc[flatid, 'State'] = 'Active'
                print(" - Active")
            else:
                title = ''.join((location,size))
                flats.loc[flatid] = [title , price, link, '--' , 'New', '']    
                print(" - New")
        else: 
            print(' - ERROR (NO INDEX)')
    
    print("Page " + str(page) + " ... Done" )
    



Page offers:21
1;0  ;;;Ul Merhautova, Brno - Brno-sever  1+1 45 m²2 660 000 Kč. https://www.sreality.cz/detail/prodej/byt/1+1/brno-brno-sever-ulice-merhautova/2056232540 - New
1;1  ;;;Ul Majdalenky, Brno - část obce Lesná  6 pokojů a více 180 m²9 069 000 Kč. https://www.sreality.cz/detail/prodej/byt/6-a-vice/brno-cast-obce-lesna-ulice-majdalenky/321560156 - New
1;2  ;;;Ul Fillova, Brno - část obce Lesná  3+1 90 m²5 190 000 Kč. https://www.sreality.cz/detail/prodej/byt/3+1/brno-cast-obce-lesna-ulice-fillova/2276064860 - New
1;3  ;;;Ul Šrámkova, Brno - část obce Lesná  2+1 55 m²3 209 000 Kč. https://www.sreality.cz/detail/prodej/byt/2+1/brno-cast-obce-lesna-ulice-sramkova/2247601756 - New
1;4  ;;;Ul Nejedlého, Brno - část obce Lesná  3+1 64 m²3 500 000 Kč. https://www.sreality.cz/detail/prodej/byt/3+1/brno-cast-obce-lesna-ulice-nejedleho/3308802396 - New
1;5  ;;;Ul Dusíkova, Brno - část obce Lesná  1+kk 46 m²2 600 000 Kč. https://www.sreality.cz/detail/prodej/byt/1+kk/brno-cast-obce-lesn

In [16]:
len(flats)

24

In [17]:
#pd.set_option('display.width', 150)
#pd.set_option('display.expand_frame_repr', True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(flats.sort_values(by='Price', ascending=True).loc[:,'Title':'Link'].to_string(index=False, justify="left"))



Title                                              Price             Link                                              
 Brno - část obce Lesná, okres Brno-město  2+1 ...      1 970 000 Kč  https://www.sreality.cz/detail/prodej/byt/2+1/...
 Ul Halasovo náměstí, Brno - část obce Lesná  1...      2 174 025 Kč  https://www.sreality.cz/detail/prodej/byt/1+1/...
 Ul Halasovo náměstí, Brno - část obce Lesná  1...      2 300 000 Kč  https://www.sreality.cz/detail/prodej/byt/1+kk...
   Ul Dusíkova, Brno - část obce Lesná  1+kk 46 m²      2 600 000 Kč  https://www.sreality.cz/detail/prodej/byt/1+kk...
       Ul Merhautova, Brno - Brno-sever  1+1 45 m²      2 660 000 Kč  https://www.sreality.cz/detail/prodej/byt/1+1/...
    Ul Milénova, Brno - část obce Lesná  2+1 60 m²      2 900 000 Kč  https://www.sreality.cz/detail/prodej/byt/2+1/...
 Ul Jurkovičova, Brno - část obce Lesná  2+1 56 m²      3 049 000 Kč  https://www.sreality.cz/detail/prodej/byt/2+1/...
    Ul Arbesova, Brno - část obce Lesná 

In [18]:
filename = "sreality_lesna"+time.strftime('%Y-%m-%d',time.gmtime())+".csv"
flats.to_csv(filename)

## Check flats based on search criteria

### **Bystrc**



In [ ]:
flats_backup = flats

pd.options.display.max_colwidth = 100

c = flats['Title'].str.contains('bystrc', case=False,regex=False)
bystrc=flats[c]

bystrc.loc[:,('Title','Price','Link')]



In [ ]:
flats.loc[1370992220, 'MyRate'] = '4'
flats.loc[1370992220, 'Comment'] = 'Velky barak - prizemi, dal od Ecerove'
flats.loc[1370992220]


In [ ]:
flats.loc[2268180060, 'MyRate'] = '8'
flats.loc[2268180060, 'Comment'] = '4p, bliz k mestu,ne velka cesta v okoli, PENB:B'
flats.loc[2268180060]


In [ ]:
flats.loc[1636408924, 'MyRate'] = '6'
flats.loc[1636408924, 'Comment'] = '3p/7, PENB:G'
flats.loc[1636408924]

In [ ]:
flats.loc[2060893788, 'MyRate'] = '6'
flats.loc[2060893788,'Comment'] = 'nadstavba 5p, vytah, drahe, mezonet (2patra), sklokeramika'
flats.loc[2060893788]

In [ ]:
flats.loc[3503472220, 'MyRate'] = '6'
flats.loc[3503472220,'Comment'] = 'divne - nekde daleko pri prehrade, PENB -neuvadi se'
flats.loc[3503472220]